## Do the _search_ on the tracker

# Selenium Part

In [2]:
import os
import configparser
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
def waitForElement(driver, by_what=By.XPATH, element_info='', delay=20, do_quit=True):
    try:
        elem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((by_what, element_info)))
        print("Page is ready!")
        return elem
    except TimeoutException:
        print("Loading took too much time!")
        print("quiting while waiting for element:", element_info)
        if do_quit:
            driver.quit()
        return None

In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')

In [4]:
DRIVER_PATH = config['DRIVER']['path']
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

In [5]:
tv_show ="westworld s03e04"
tv_show = tv_show.replace(' ', '+')

driver.get('https://1337x.gd/search/' + tv_show + '/1/')
waitForElement(driver, element_info="//div[contains(@class,'table-list-wrap')]")

Page is ready!


<selenium.webdriver.remote.webelement.WebElement (session="3b73af4650eaad17b05a281533ee43ec", element="556234ac-607c-4f15-a5bb-d612ba95d299")>

Passing to beautifulsoup

In [6]:
raw_page = BeautifulSoup(driver.page_source, 'html.parser')
# driver.close()

Creating table header:

In [36]:
a = raw_page.find_all('tr')[0]

row_marker = 0
column_marker = 0
col_names = []
for b in a.find_all('th'):
    col_names.append(b.get_text())
    column_marker += 1

col_names.append('url')  # assuming there's always some urls
print(col_names)


['name', 'se', 'le', 'time', 'size info', 'uploader', 'url']


In [37]:
def getTorrentLink(cell):
    for x in cell.find_all('a', href=True):
        if 'torrent/' in x['href']:
            return x['href']

In [38]:
urls = []
names = []
ses = []
les = []
times = []
sizes = []
uploaders = []

for row in raw_page.find_all('tr'):
#     print('row: ', row)
    max_col = len(row.find_all('td'))
    i = 0
    for col in row.find_all('td'):
        if i == 0:
            urls.append(getTorrentLink(col))
#             print('url: ', getTorrentLink(col))
            names.append(col.get_text())
#             print('name: ', col.get_text())
        if i == 1:
            ses.append(col.get_text())
#             print('se: ', col.get_text())
        if i == 2:
            les.append(col.get_text())
#             print('le: ', col.get_text())
        if i == 3:
            times.append(col.get_text())
#             print('time: ', col.get_text())
        if i == 4:
            sizes.append(col.get_text())
#             print('size info: ', col.get_text())
        if i == 5:
            uploaders.append(col.get_text())
#             print('uploader: ', col.get_text())
        i += 1
#     print()

In [39]:
curTorrentTable = pd.DataFrame(columns=col_names)

In [40]:
# print(curTorrentTable)

In [41]:
curTorrentTable['name'] = names
curTorrentTable['se'] = ses
curTorrentTable['le'] = les
curTorrentTable['time'] = times
curTorrentTable['size'] = sizes
curTorrentTable['uploader'] = uploaders
curTorrentTable['url'] = urls

In [42]:
print(curTorrentTable)

                                                 name    se   le     time  \
0           Westworld S03E04 720p WEB H264-XLF [eztv]  1419  427   3:03am   
1          Westworld.S03E04.1080p.WEB.H264-XLF[TGx] ⭐   974  399   3:11am   
2   Westworld S03E04 The Mother of Exiles 720p AMZ...   823  206   3:32am   
3                Westworld S03E04 WEB H264-XLF [eztv]   763  238   3:02am   
4         Westworld.S03E04.720p.WEB.x265-MiNX[TGx] ⭐1   606  190   3:36am   
5   Westworld.S03E04.The.Mother.of.Exiles.1080p.AM...   489  170   3:32am   
6                 Westworld.S03E04.1080p.WEB.H264-XLF   401  231   7:40am   
7               Westworld S03E04 480p x264-mSD [eztv]   385   92   3:11am   
8   Westworld.S03E04.The.Mother.of.Exiles.1080p.AM...   359  374   7:40am   
9                 Westworld.S03E04.WEB.H264-XLF[ettv]   378   61   3:05am   
10                 Westworld.S03E04.720p.WEB.H264-XLF   296  107   7:39am   
11          Westworld.S03E04.720p.WEB.H264-XLF[TGx] ⭐   151   15   3:03am   

Casting types:

In [43]:
test = curTorrentTable.copy()
test['size'] = test['size'].replace({r' GB[0-9]+' : r''}, regex=True)
test['size'] = test['size'].apply(lambda x: x if ('MB' in x) else float(x)*1024)
test['size'] = test['size'].replace({r' MB[0-9]+' : r''}, regex=True)
test['size'] = test['size'].astype(float)

test['se'] = test['se'].astype(int)
test['le'] = test['le'].astype(int)
test['time'] = pd.to_datetime(test['time'])

In [44]:
# good_links = test.loc[test.name.str.contains('1080p')]['url']

In [45]:
# test.loc[test.name.str.contains('1080p')]['url'].iloc[0]

'/torrent/4393086/Westworld-S03E04-1080p-WEB-H264-XLF-TGx/'

In [46]:
# indexNames = test[ ('720p' in test['name'])].index
# test.drop(indexNames , inplace=False)

KeyError: False

In case we want to find only urls from the page

## Block for choosing the best link:

In [ ]:
# curTorrentTable['size'] = 



In [47]:
print(test)
print(test.describe())

                                                 name    se   le  \
0           Westworld S03E04 720p WEB H264-XLF [eztv]  1419  427   
1          Westworld.S03E04.1080p.WEB.H264-XLF[TGx] ⭐   974  399   
2   Westworld S03E04 The Mother of Exiles 720p AMZ...   823  206   
3                Westworld S03E04 WEB H264-XLF [eztv]   763  238   
4         Westworld.S03E04.720p.WEB.x265-MiNX[TGx] ⭐1   606  190   
5   Westworld.S03E04.The.Mother.of.Exiles.1080p.AM...   489  170   
6                 Westworld.S03E04.1080p.WEB.H264-XLF   401  231   
7               Westworld S03E04 480p x264-mSD [eztv]   385   92   
8   Westworld.S03E04.The.Mother.of.Exiles.1080p.AM...   359  374   
9                 Westworld.S03E04.WEB.H264-XLF[ettv]   378   61   
10                 Westworld.S03E04.720p.WEB.H264-XLF   296  107   
11          Westworld.S03E04.720p.WEB.H264-XLF[TGx] ⭐   151   15   
12                   Westworld.S03E04.XviD-AFG[TGx] ⭐    87   29   
13               Westworld.S03E04.WEB.H264-XLF[T

In [48]:
best_torrent = test.loc[test.name.str.contains('1080p')]['url'].iloc[0] # TODO: add better choosing technique

## Pulling out a magnet link:

In [49]:
# DRIVER_PATH = '/Users/Alexey/PycharmProjects/Scrapping/chromedriver'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)
best_torrent_link = "https://1337x.gd" + best_torrent
driver.get(best_torrent_link)
waitForElement(driver, element_info="//div[contains(@class,'search-box')]")

Page is ready!


<selenium.webdriver.remote.webelement.WebElement (session="23a9a6da9941cc73776f9baca50ea892", element="e415892e-e2b7-4ec3-b10b-e573b4e7d5f3")>

In [50]:
import time

final_page = BeautifulSoup(driver.page_source, 'html.parser')
driver.close()  # IMPORTANT
# print(final_page.prettify())

extracting magnet:

In [51]:
magnet_link = str()
for a in final_page.find_all('a', href=True):
    if 'magnet' in a['href']:
        magnet_link = a['href']
        break

and adding it to transmission-remote with: `system.os('transmission-remote -a "' + magnet link + '"')`

In [52]:
print('transmission-remote -a "' + magnet_link +'"')

transmission-remote -a "magnet:?xt=urn:btih:34F466B0ACD1220C61CE3F7DBECC176B8E1B4935&dn=Westworld.S03E04.1080p.WEB.H264-XLF%5BTGx%5D+%E2%AD%90&tr=udp%3A%2F%2Ftracker.coppersurfer.tk%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.leechers-paradise.org%3A6969%2Fannounce&tr=udp%3A%2F%2Fopen.stealth.si%3A80%2Fannounce&tr=udp%3A%2F%2Fexodus.desync.com%3A6969&tr=udp%3A%2F%2Fp4p.arenabg.com%3A1337%2Fannounce&tr=udp%3A%2F%2Fexplodie.org%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.tiny-vps.com%3A6969%2Fannounce&tr=udp%3A%2F%2Fopen.demonii.si%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.torrent.eu.org%3A451%2Fannounce&tr=udp%3A%2F%2Ftracker.pirateparty.gr%3A6969%2Fannounce&tr=udp%3A%2F%2Fipv4.tracker.harry.lu%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.cyberia.is%3A6969%2Fannounce&tr=udp%3A%2F%2F9.rarbg.to%3A2710%2Fannounce&tr=udp%3A%2F%2Ftracker.zer0day.to%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.leechers-paradise.org%3A6969%2Fannounce&tr=udp%3A%2F%2Fcopper